In [126]:
import numpy as np
import random

from strawberryfields.decompositions import williamson, bloch_messiah

from src.gbs_matrices import GraphMatrices, GaussianMatrices
from src.symplectic import Symplectic, SymplecticFock, SymplecticXXPP

from strawberryfields.decompositions import takagi

In [12]:
import matplotlib.pyplot as plt

In [324]:
def make_random_graph(M, max_degree):
    """this is just a different way of doing what you already did, but I wanted to have a go"""
    B = np.zeros((M, M), dtype=int)

    all_vertices = np.arange(M)
    np.random.shuffle(all_vertices)

    for i, start_vertex in enumerate(all_vertices):
        end_vertices = all_vertices[i:]
        start_vertex_degree = B[start_vertex,:].sum()
        possible_edge_ends = np.random.choice(end_vertices, min(max_degree-start_vertex_degree, M-i-1), replace=False)
        for possible_edge_end in possible_edge_ends:
            end_vertex_degree = B[possible_edge_end,:].sum()
            if end_vertex_degree < max_degree:
                B[start_vertex, possible_edge_end] = 1
                B[possible_edge_end, start_vertex] = 1
    
    return B

In [ ]:
def squeezing_params(B):
    

In [311]:
# <<<<<<<<<<<<<<<<<<< Basic parameters  >>>>>>>>>>>>>>>>>>
M = 6  # Number of modes
max_degree = 4  # < M - 1
x = - 1 / 4*(max_degree - 1) - 1  # smaller than - 1 / 4*(max_degree - 1)

In [312]:
adj_matrix = make_random_graph(M, max_degree)

In [313]:
g_matrix = x * adj_matrix

In [316]:
def get_squeeze_params(gamma_vec):
    B = g_matrix * np.outer(gamma_vec, gamma_vec)
    eigsB = np.linalg.eigvals(B)
    inv_c = max(eigsB, key=abs)
    cB = 0.5 * B / inv_c
    
    squeezing = np.arctanh(takagi(cB)[0])
    return squeezing

In [317]:
def cost(gamma_vec):
    squeezing = get_squeeze_params(gamma_vec)
    return max(squeezing) - min(squeezing)

In [318]:
from scipy.optimize import minimize

In [319]:
gamma_vec_guess = 0.1 * np.ones(M)
result = minimize(cost, x0=gamma_vec_guess)

optimised_gamma_vec = result.x

In [320]:
optimised_gamma_vec

array([0.08306941, 0.08306941, 0.10213551, 0.11440283, 0.14393728,
       0.07546185])

In [321]:
get_squeeze_params(optimised_gamma_vec)

array([0.54930614, 0.27130726, 0.20908804, 0.1266861 , 0.10527339,
       0.10527339])

In [322]:
get_squeeze_params(gamma_vec_guess)

array([0.54930614, 0.24069262, 0.18816834, 0.13846887, 0.11560946,
       0.07819936])

In [323]:
print(cost(gamma_vec_guess))
print(cost(optimised_gamma_vec))

0.47110678548393997
0.4440327578899577
